In [201]:
import matplotlib.pyplot as plt # for plotting maps
import maup # mggg's library for proration, see documentation here: https://github.com/mggg/maup
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
from statistics import mean, median
from functools import reduce

**Load Election Results and Turnout Data**

Importing the results from a file that connects the elections to 2016 VTDs

In [244]:
returns_2012 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2012_General_Election_Returns.csv')
returns_2014 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2014_General_Election_Returns.csv')
returns_2016 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2016_General_Election_Returns.csv')

print(returns_2012.head())

     County  FIPS   VTD cntyvtd     Office    Name Party Incumbent  Votes
0  Anderson     1  0001   10001  President  Romney     R         N    754
1  Anderson     1  0002   10002  President  Romney     R         N   1179
2  Anderson     1  0003   10003  President  Romney     R         N    650
3  Anderson     1  0004   10004  President  Romney     R         N    729
4  Anderson     1  0005   10005  President  Romney     R         N    174


In [203]:
VRTO_2012 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2012_General_Election_VRTO.csv')
VRTO_2014 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2014_General_Election_VRTO.csv')
VRTO_2016 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2016_General_Election_VRTO.csv')

**Load VTD Shapefile**

2016 VTD Shapefile is used for these results

In [204]:
vtd_2016 = gp.read_file('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/vtd16g/vtd16g.shp')

**Load Census Data**

2010 Census Data tied to 2016 VTDs

In [241]:
census_data = pd.read_excel('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/Pop2010_VTD_2016G.xlsx')
print(census_data.head())

   CNTY   VTD  CNTYVTD  e_ang  e_oth  e_hsp  e_total  e_vap  e_blak  e_bh  \
0    53  0008   530008   1117     19    131     1269    993       2   133   
1   469  0024  4690024   1357      8    276     1679   1260      38   314   
2   421  0302  4210302    372     10    125      510    364       3   128   
3   479  0340  4790340    336     87   2839     3274   2460      17  2851   
4    29  4046   294046   2038    225   1012     3644   3083     416  1381   

   e_angvap  e_hspvap  e_bhvap  e_blakvap  e_othvap  ep_ang  ep_oth  ep_hsp  \
0       901        74       76          2        16   0.880   0.015   0.103   
1      1044       179      209         30         7   0.808   0.005   0.164   
2       272        86       87          1         5   0.729   0.020   0.245   
3       275      2111     2118         10        67   0.103   0.027   0.867   
4      1839       778     1057        301       187   0.559   0.062   0.278   

   ep_blak  ep_bh  ep_angvap  ep_hspvap  ep_bhvap  ep_blakvap 

**Load TX-House, TX-Sen, and US-House info**

In [243]:
us_house = gp.read_file('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Original Files/house_sen/tl_2016_us_cd115/tl_2016_us_cd115.shp')
tx_sen = gp.read_file('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Original Files/house_sen/PLANS172/PLANS172.shp')
tx_house = gp.read_file('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Original Files/house_sen/PLANH407/PLANH407.shp')

tx_sen.index = np.arange(1, len(tx_sen)+1)
tx_house.index = np.arange(1, len(tx_house)+1)


   District                                           geometry
1         1  POLYGON ((1437721.638 1208032.925, 1437711.322...
2         2  POLYGON ((1368617.272 1138307.869, 1368617.076...
3         3  POLYGON ((1383052.514 847694.619, 1383021.487 ...
4         4  POLYGON ((1372124.540 1099497.361, 1372124.382...
5         5  POLYGON ((1449163.877 1116990.631, 1449141.632...


**Pivot Election Data**

In [248]:
office_list_2012 = ["U.S. Sen","President"]
party_list = ["R","D"]
returns_2012_filtered = returns_2012[returns_2012['Office'].isin(office_list_2012)]
returns_2012_filtered = returns_2012_filtered[returns_2012_filtered['Party'].isin(party_list)]
pivoted_2012 = pd.pivot_table(returns_2012_filtered, values=['Votes'], index=["cntyvtd","County"],columns=["Office","Party"])
print(pivoted_2012.head())
pivoted_2012.columns = ["PRES12D","PRES12R","SEN12D","SEN12R"]
pivoted_2012.index.names = ["CNTYVTD","COUNTY"]
#print(pivoted_2012.head())
#print(pivoted_2012.count())

                     Votes                     
Office           President       U.S. Sen      
Party                    D     R        D     R
cntyvtd County                                 
10001   Anderson       261   754      285   713
10002   Anderson       228  1179      274  1072
10003   Anderson       108   650      126   607
10004   Anderson       128   729      155   675
10005   Anderson        13   174       29   152


In [209]:
office_list_2014 = ["U.S. Sen","Governor"]
party_list = ["R","D"]
returns_2014_filtered = returns_2014[returns_2014['Office'].isin(office_list_2014)]
returns_2014_filtered = returns_2014_filtered[returns_2014_filtered['Party'].isin(party_list)]
pivoted_2014 = pd.pivot_table(returns_2014_filtered, values=['Votes'], index=["cntyvtd"],columns=["Office","Party"])
pivoted_2014.columns = ["GOV14D","GOV14R","SEN14D","SEN14R"]
pivoted_2014.index.names = ["CNTYVTD"]
#print(pivoted_2014.head())
#print(pivoted_2014.count())

In [210]:
office_list_2016 = ["President"]
party_list = ["R","D"]
returns_2016_filtered = returns_2016[returns_2016['Office'].isin(office_list_2016)]
returns_2016_filtered = returns_2016_filtered[returns_2016_filtered['Party'].isin(party_list)]
pivoted_2016 = pd.pivot_table(returns_2016_filtered, values=['Votes'], index=["cntyvtd"],columns=["Office","Party"])
pivoted_2016.columns = ["PRES16D","PRES16R"]
pivoted_2016.index.names = ["CNTYVTD"]
#print(pivoted_2016.head())
#print(pivoted_2016.count())

**Clean Turnout Data**

In [211]:
VRTO_2012_filtered = VRTO_2012[["CNTYVTD","TotalVR","TotalTO"]]
VRTO_2012_filtered = VRTO_2012_filtered.rename(columns={"TotalVR": "TOTVR12", "TotalTO": "TOTTO12"})
VRTO_2014_filtered = VRTO_2014[["CNTYVTD","TotalVR","TotalTO"]]
VRTO_2014_filtered = VRTO_2014_filtered.rename(columns={"TotalVR": "TOTVR14", "TotalTO": "TOTTO14"})
VRTO_2016_filtered = VRTO_2016[["CNTYVTD","TotalVR","TotalTO"]]
VRTO_2016_filtered = VRTO_2016_filtered.rename(columns={"TotalVR": "TOTVR16", "TotalTO": "TOTTO16"})

**Clean Census Data**

In [212]:
census_data = census_data[["VTD","CNTYVTD","e_ang","e_oth","e_hsp","e_total","e_vap","e_blak","e_bh","e_angvap","e_hspvap","e_bhvap","e_blakvap","e_othvap"]]

**Combine Turnout, Election, and Census Data**

In [213]:
all_voter_data = [census_data,pivoted_2012,VRTO_2012_filtered,pivoted_2014,VRTO_2014_filtered,pivoted_2016,VRTO_2016_filtered,vtd_2016]
merged_voter_data = reduce(lambda  left,right: pd.merge(left,right,on=['CNTYVTD'], how='outer'), all_voter_data)
print(merged_voter_data.count())

VTD_x         8941
CNTYVTD       8941
e_ang         8941
e_oth         8941
e_hsp         8941
e_total       8941
e_vap         8941
e_blak        8941
e_bh          8941
e_angvap      8941
e_hspvap      8941
e_bhvap       8941
e_blakvap     8941
e_othvap      8941
PRES12D       8941
PRES12R       8941
SEN12D        8941
SEN12R        8941
TOTVR12       8941
TOTTO12       8941
GOV14D        8941
GOV14R        8941
SEN14D        8941
SEN14R        8941
TOTVR14       8941
TOTTO14       8941
PRES16D       8941
PRES16R       8941
TOTVR16       8941
TOTTO16       8941
CNTY          8941
COLOR         8941
VTD_y         8941
VTDKEY        8941
Shape_area    8941
Shape_len     8941
geometry      8941
dtype: int64


**Assign to House Districts**

In [214]:
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
merged_voter_data = gp.GeoDataFrame(merged_voter_data, geometry='geometry')

proj = vtd_2016.crs
vtd_2016 = vtd_2016.to_crs(proj)

merged_voter_data.set_crs(proj)
merged_voter_data["geometry"] = merged_voter_data.buffer(0)

tx_sen.set_crs(proj)
tx_sen["geometry"] = tx_sen.buffer(0)

tx_house.set_crs(proj)
tx_house["geometry"] = tx_house.buffer(0)

us_house = us_house.to_crs(epsg=3081)
us_house["geometry"] = us_house.buffer(0)

us_house = us_house[us_house['STATEFP']=='48']
print(us_house.head())
print(us_house.crs)
print(us_house.count())

    STATEFP CD115FP GEOID                  NAMELSAD LSAD CDSESSN  MTFCC  \
44       48      01  4801  Congressional District 1   C2     115  G5200   
87       48      02  4802  Congressional District 2   C2     115  G5200   
125      48      03  4803  Congressional District 3   C2     115  G5200   
161      48      04  4804  Congressional District 4   C2     115  G5200   
191      48      05  4805  Congressional District 5   C2     115  G5200   

    FUNCSTAT        ALAND      AWATER     INTPTLAT      INTPTLON  \
44         N  20379602044   951776362  +31.9516781  -094.5520460   
87         N    799720185    54789657  +30.0473686  -095.1782923   
125        N   1245661528    97740060  +33.1594063  -096.6130717   
161        N  26237731127  1010312804  +33.2982880  -095.4180633   
191        N  13064499072   494023014  +32.2257951  -095.7316676   

                                              geometry  
44   POLYGON ((1412302.784 1176455.798, 1412346.279...  
87   POLYGON ((1419867.628

In [215]:
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
tx_sen_assign = maup.assign(merged_voter_data, tx_sen)
tx_house_assign = maup.assign(merged_voter_data, tx_house)


In [216]:
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
us_house.index = np.arange(1, len(us_house)+1)
us_house_assign = maup.assign(merged_voter_data, us_house)

In [218]:
print(tx_sen_assign.head())
print(tx_house_assign.head())
print(us_house_assign.head())

merged_voter_data['USCD']=us_house_assign
merged_voter_data['SEND']=tx_sen_assign
merged_voter_data['HD']=tx_house_assign

print(merged_voter_data.head())

0    24
1    18
2    31
3    21
4    25
dtype: int64
0     20
1     30
2     87
3     42
4    121
dtype: int64
0    25
1    27
2    13
3    28
4    21
dtype: int64
  VTD_x  CNTYVTD  e_ang  e_oth  e_hsp  e_total  e_vap  e_blak  e_bh  e_angvap  \
0  0008   530008   1117     19    131     1269    993       2   133       901   
1  0024  4690024   1357      8    276     1679   1260      38   314      1044   
2  0302  4210302    372     10    125      510    364       3   128       272   
3  0340  4790340    336     87   2839     3274   2460      17  2851       275   
4  4046   294046   2038    225   1012     3644   3083     416  1381      1839   

   ...  CNTY  COLOR  VTD_y  VTDKEY    Shape_area      Shape_len  \
0  ...    53      2   0008       0  6.717692e+07   55772.006345   
1  ...   469      2   0024       0  5.426296e+07   50831.107001   
2  ...   421      6   0302       0  4.890835e+08  115435.448089   
3  ...   479      1   0340       0  1.949293e+06    5991.528062   
4  ...    29  

**Rename Columns to Match PGOP**

In [249]:
merged_voter_data = merged_voter_data.rename(columns={"TotalVR": "TOTVR16", "TotalTO": "TOTTO16"})
merged_voter_data = merged_voter_data.rename(columns={"e_ang":"WHITE","e_oth":"OTHER","e_hsp":"HISPANIC","e_total":"TOTPOP","e_vap":"VAP","e_blak":"BLACK","e_bh":"BLKHISP","e_angvap":"WVAP","e_hspvap":"HISPVAP","e_bhvap":"BHVAP","e_blakvap":"BVAP","e_othvap":"OTHVAP","CNTY":"FIPS"})
print(merged_voter_data.count())

VTD_x         8941
CNTYVTD       8941
WHITE         8941
OTHER         8941
HISPANIC      8941
TOTPOP        8941
VAP           8941
BLACK         8941
BLKHISP       8941
WVAP          8941
HISPVAP       8941
BHVAP         8941
BVAP          8941
OTHVAP        8941
PRES12D       8941
PRES12R       8941
SEN12D        8941
SEN12R        8941
TOTVR12       8941
TOTTO12       8941
GOV14D        8941
GOV14R        8941
SEN14D        8941
SEN14R        8941
TOTVR14       8941
TOTTO14       8941
PRES16D       8941
PRES16R       8941
TOTVR16       8941
TOTTO16       8941
FIPS          8941
COLOR         8941
VTD_y         8941
VTDKEY        8941
Shape_area    8941
Shape_len     8941
geometry      8941
USCD          8941
SEND          8941
HD            8941
dtype: int64


### Load PGP TX shapefile to compare results

In [221]:
tx_pgp_output = gp.read_file('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/TX_vtds/TX_vtds.shp')

In [227]:
print(tx_pgp_output.count())
print(tx_pgp_output.head())
print(tx_pgp_output[["FIPS","CNTYVTD"]].head(30))

CNTYVTD     8941
VTD         8941
WHITE       8941
OTHER       8941
HISPANIC    8941
TOTPOP      8941
VAP         8941
BLACK       8941
BLKHISP     8941
WVAP        8941
HISPVAP     8941
BHVAP       8941
BVAP        8941
OTHVAP      8941
COUNTY      8941
FIPS        8941
PRES12R     8941
PRES12D     8941
SEN12R      8941
SEN12D      8941
TOTVR12     8941
TOTTO12     8941
SEN14R      8941
SEN14D      8941
GOV14R      8941
GOV14D      8941
TOTVR14     8941
TOTTO14     8941
PRES16D     8941
PRES16R     8941
TOTVR16     8941
TOTTO16     8941
USCD        8941
SEND        8941
HD          8941
AREA        8941
PERIM       8941
geometry    8941
dtype: int64
  CNTYVTD   VTD   WHITE  OTHER  HISPANIC  TOTPOP     VAP  BLACK  BLKHISP  \
0   10001  0001  2053.0   89.0     401.0  3131.0  2341.0  606.0    989.0   
1   10002  0002  3171.0   64.0     245.0  3744.0  2732.0  272.0    509.0   
2   10003  0003  1443.0   32.0     151.0  1766.0  1345.0  140.0    291.0   
3   10004  0004  1962.0   40.0      7

In [240]:
# Joining my version and the PGP final version
validate = pd.merge(merged_voter_data, tx_pgp_output, on=['CNTYVTD'], how='left')
pd.set_option('display.max_columns', None)
display(validate)

,VTD_x,CNTYVTD,WHITE_x,OTHER_x,HISPANIC_x,TOTPOP_x,VAP_x,BLACK_x,BLKHISP_x,WVAP_x,HISPVAP_x,BHVAP_x,BVAP_x,OTHVAP_x,PRES12D_x,PRES12R_x,SEN12D_x,SEN12R_x,TOTVR12_x,TOTTO12_x,GOV14D_x,GOV14R_x,SEN14D_x,SEN14R_x,TOTVR14_x,TOTTO14_x,PRES16D_x,PRES16R_x,TOTVR16_x,TOTTO16_x,CNTY,COLOR,VTD_y,VTDKEY,Shape_area,Shape_len,geometry_x,USCD_x,SEND_x,HD_x,VTD,WHITE_y,OTHER_y,HISPANIC_y,TOTPOP_y,VAP_y,BLACK_y,BLKHISP_y,WVAP_y,HISPVAP_y,BHVAP_y,BVAP_y,OTHVAP_y,COUNTY,FIPS,PRES12R_y,PRES12D_y,SEN12R_y,SEN12D_y,TOTVR12_y,TOTTO12_y,SEN14R_y,SEN14D_y,GOV14R_y,GOV14D_y,TOTVR14_y,TOTTO14_y,PRES16D_y,PRES16R_y,TOTVR16_y,TOTTO16_y,USCD_y,SEND_y,HD_y,AREA,PERIM,geometry_y
0,0008,530008,1117,19,131,1269,993,2,133,901,74,76,2,16,121,520,133,494,921,668,94,350,75,366,959,470,127,611,1092,779,53,2,0008,0,6.717692e+07,55772.006345,"POLYGON ((1159123.585 956882.966, 1159183.604 ...",25,24,20,0008,1117.0,19.0,131.0,1269.0,993.0,2.0,133.0,901.0,74.0,76.0,2.0,16.0,Burnet,53,520,121,494,133,921,668,366,75,350,94,959,470,127,611,1092,779,25,24,20,68,56,"POLYGON ((1159123.585 956882.966, 1159183.604 ..."
1,0024,4690024,1357,8,276,1679,1260,38,314,1044,179,209,30,7,109,735,117,694,1153,854,77,505,60,503,1272,587,107,845,1338,977,469,2,0024,0,5.426296e+07,50831.107001,"POLYGON ((1292779.280 755618.564, 1292761.756 ...",27,18,30,0024,1357.0,8.0,276.0,1679.0,1260.0,38.0,314.0,1044.0,179.0,209.0,30.0,7.0,Victoria,469,735,109,694,117,1153,854,503,60,505,77,1272,587,107,845,1338,977,27,18,30,55,51,"POLYGON ((1292779.280 755618.564, 1292761.756 ..."
2,0302,4210302,372,10,125,510,364,3,128,272,86,87,1,5,15,146,16,134,216,162,3,76,2,76,228,78,11,126,226,138,421,6,0302,0,4.890835e+08,115435.448089,"POLYGON ((835910.171 1592476.454, 835993.315 1...",13,31,87,0302,372.0,10.0,125.0,510.0,364.0,3.0,128.0,272.0,86.0,87.0,1.0,5.0,Sherman,421,146,15,134,16,216,162,76,2,76,3,228,78,11,126,226,138,13,31,87,489,115,"POLYGON ((835910.171 1592476.454, 835993.315 1..."
3,0340,4790340,336,87,2839,3274,2460,17,2851,275,2111,2118,10,67,785,673,644,742,2049,1484,404,390,337,394,2197,821,759,554,2221,1360,479,1,0340,0,1.949293e+06,5991.528062,"POLYGON ((1052058.489 602069.618, 1052032.480 ...",28,21,42,0340,336.0,87.0,2839.0,3274.0,2460.0,17.0,2851.0,275.0,2111.0,2118.0,10.0,67.0,Webb,479,673,785,742,644,2049,1484,394,337,390,404,2197,821,759,554,2221,1360,28,21,42,2,6,"POLYGON ((1052058.489 602069.618, 1052032.480 ..."
4,4046,294046,2038,225,1012,3644,3083,416,1381,1839,778,1057,301,187,581,939,579,895,2215,1561,355,625,278,678,2314,998,664,798,2506,1589,29,5,4046,0,2.356032e+06,6851.937823,"POLYGON ((1151858.098 818462.053, 1151860.759 ...",21,25,121,4046,2038.0,225.0,1012.0,3644.0,3083.0,416.0,1381.0,1839.0,778.0,1057.0,301.0,187.0,Bexar,29,939,581,895,579,2215,1561,678,278,625,355,2314,998,664,798,2506,1589,21,25,121,2,7,"POLYGON ((1151858.098 818462.053, 1151860.759 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8936,0451,2010451,2853,127,321,3444,2927,157,464,2496,225,326,109,105,333,1713,325,1702,2653,2088,268,1208,182,1247,2684,1492,433,1536,2783,2039,201,7,0451,0,3.798154e+06,9207.399557,"POLYGON ((1430674.146 879003.969, 1430697.504 ...",2,7,126,0451,2853.0,127.0,321.0,3444.0,2927.0,157.0,464.0,2496.0,225.0,326.0,109.0,105.0,Harris,201,1713,333,1702,325,2653,2088,1247,182,1208,268,2684,1492,433,1536,2783,2039,02,7,126,4,9,"POLYGON ((1430674.146 879003.969, 1430697.504 ..."
8937,0623,2010623,3370,545,1181,6128,4859,1076,2213,2910,820,1539,743,410,780,1503,752,1470,3490,2337,405,936,347,933,3736,1360,888,1378,3991,2374,201,1,0623,0,4.948858e+06,15892.709947,"POLYGON ((1431613.134 881072.025, 1431655.786 ...",2,7,126,0623,3370.0,545.0,1181.0,6128.0,4859.0,1076.0,2213.0,2910.0,820.0,1539.0,743.0,410.0,Harris,201,1503,78

In [232]:
validate['pop_diff'] = validate.TOTPOP_x - validate.TOTPOP_y
validate[validate.pop_diff.abs()<10].shape[0] / validate.shape[0]

1.0

In [233]:
validate['pres12_diff'] = validate.PRES12R_x - validate.PRES12R_y
validate[validate.pres12_diff.abs()<10].shape[0] / validate.shape[0]

1.0

In [245]:
#To do, add in County name info